<a href="https://colab.research.google.com/github/kirkwilson/crop_images/blob/main/crop_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

done
- user set crop variables
- upload zip of images

todo:
- unzip images
- crop images
- compress new images
- clean up old files
- write readme

compressed files

Input will likely be a compressed folder of images. Planning to support both zip
and 7z files. Will need to iterate through the `contents` folder and decompress.

Running dict of files to crop. Need to decompress and walk files.

In [ ]:
#@markdown (x, y) pixel coordinates of the top left crop rectangle.
crop_anchor_x = 7 #@param {type:"number"}
crop_anchor_y = 0 #@param {type:"number"}
#@markdown ---
#@markdown Crop area in pixels from `crop_anchor_y` down.
crop_height = 10 #@param {type:"number"}
#@markdown Crop area in pixels from `crop_anchor_x` right.
crop_width = 10 #@param {type:"number"}

In [ ]:
from collections import namedtuple
from google.colab import files
import os
from PIL import Image
from zipfile import ZipFile

IMAGE_FILE_EXT = ['.png', '.jpg']

def is_int(input) -> int:
  if not isinstance(input, int):
    raise TypeError(f'{input} is not an integer.')
  return input

def is_positive(input) -> int:
  if is_int(input):
    if input < 0:
      raise ValueError(f'{input} is not a positive number.')
  return input

def validate_input(input) -> int:
  is_int(input)
  is_positive(input)
  return input

def validate_coordinates(x, y) -> tuple[int, int]:
  validate_input(x)
  validate_input(y)
  return (x, y)

def get_point(x, y) -> tuple[int, int]:
  validate_coordinates(x, y)
  Point = namedtuple('Point', ['x', 'y'])
  return Point(x, y)

def upload_files() -> list[str]:
  uploaded = files.upload()
  return list(uploaded.keys())

def is_existing_file(path) -> tuple[str, bool, bool]:
  IsExistingFile = namedtuple('ExistingFile', ['path', 'exists', 'is_file'])
  return IsExistingFile(path, os.path.exists(path), os.path.isfile(path))

def validate_is_existing_file(IsExistingFile) -> tuple[str, bool, bool]:
  if not IsExistingFile.path:
    error_message = ('No file path given. '
                     'A path is required when creating a valid IsExistingFile.')
    raise ValueError(error_message)
  if not IsExistingFile.exists:
    error_message = (f'Could not find "{IsExistingFile.path}". '
                     'An existing file is requried for a valid IsExistingFile.')
    raise FileNotFoundError(error_message)
  if not IsExistingFile.is_file:
    error_message = (f'"{IsExistingFile.path}" is not a file. '
                    'File required to create a valid IsExistingFile.')
    raise TypeError(error_message)
  return IsExistingFile

def is_valid_existing_file(path) -> tuple[str, bool, bool]:
  IsExistingFile = is_existing_file(path)
  validate_is_existing_file(IsExistingFile)
  return IsExistingFile

def split_basename(path) -> dict[dict[str, str]]:
  if is_valid_existing_file(path):
    filename, file_extension = os.path.splitext(path)
    file_dict = {
        path : {
            'filename': filename,
            'file_extension': file_extension}}
  return file_dict

def decompress_zipfile(path) -> list[str]:
  uploaded_file = split_basename(path)
  if uploaded_file[path]['file_extension'] == '.zip':
    with ZipFile(path, 'r') as zip_ref:
      extracted_files = zip_ref.namelist()
      zip_ref.extractall()
    os.remove(path)
  else:
    raise TypeError(f'"{path}" is not a .zip file')
  return extracted_files

def decompress_files(file_list) -> list[str]:
  decompressed_files = []
  for upload in file_list:
    ExistingFile = is_existing_file(upload)
    uploaded_file = split_basename(ExistingFile.path)
    if uploaded_file[upload]['file_extension'] == '.zip':
      decompressed = decompress_zipfile(upload)
      decompressed_files = decompressed_files + decompressed
      file_list.remove(upload)
  return file_list + decompressed_files

def is_image_file(path, valid_types) -> dict[dict[str, str]]:
  if is_valid_existing_file(path):
    uploaded_file = split_basename(path)
    if not uploaded_file[path]['file_extension'] in valid_types:
      error_message = (f'"{path}" must be one of these types: {valid_types}.')
  return uploaded_file

def crop_image(path, valid_types, crop_box) -> str:
  valid_image = is_image_file(path, valid_types)
  if valid_image:
    img = Image.open(path)
    cropped = img.crop((left, top, right, bottom))
    file_name = valid_image[path]['filename']
    file_extension = valid_image[path]['file_extension']
    cropped_file_path = (f'{file_name}_cropped{file_extension}')
    cropped.save(cropped_file_path)
  return cropped_file_path

def crop_images(file_list, valid_types, crop_box) -> list[str]:
  cropped_images = []
  for upload in file_list:
    IsExistingFile = is_existing_file(upload)
    if IsExistingFile.is_file:
      uploaded_file = split_basename(upload)
      if uploaded_file[upload]['file_extension'] in valid_types:
        cropped_path = crop_image(upload, valid_types, crop_box)
        cropped_images.append(cropped_path)
  return cropped_images

In [ ]:
left = crop_anchor_x
top = crop_anchor_y
right = crop_anchor_x + crop_width
bottom = crop_anchor_y + crop_height
crop_box = (left, top, right, bottom)

uploaded = upload_files()

decompressed_files = decompress_files(uploaded)

cropped_images = crop_images(decompressed_files, IMAGE_FILE_EXT, crop_box)